# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data

In [ ]:
business_df = pd.read_csv("/content/drive/MyDrive/Corelli/Duke Spring 2022/AIPI 540/Recommendation Systems Module Project/data/business_df_final.csv")

In [ ]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,open_past_10
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...",0
1,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G...",1
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",0
3,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food",1
4,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127118,hn9Toz3s-Ei3uZPt7esExA,West Side Kebab House,2470 Guardian Road NW,Edmonton,AB,T5T 1K8,53.509649,-113.675999,4.5,18,0,"Middle Eastern, Restaurants",1
127119,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"Nail Salons, Beauty & Spas",0
127120,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"Pets, Nurseries & Gardening, Pet Stores, Hobby...",0
127121,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"Fitness/Exercise Equipment, Eyewear & Optician...",0


# Grab 50,000 examples

In [ ]:
business_df = business_df.sample(n=50000)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,open_past_10
104262,gczLFZVmpfsBB3uUHEbWiA,HipCityVeg,121 S Broad St,Philadelphia,PA,19107,39.950265,-75.163685,4.5,157,1,"Vegan, Breakfast & Brunch, Vegetarian, Restaur...",0
65693,4n29_s1qDN5HQAYst74ClA,Always Open & Shut Garage Door Services,"4500 E Speedway Blvd, Ste 95",Tucson,AZ,85712,32.235482,-110.895939,4.0,28,1,"Contractors, Home Services, Shopping, Handyman...",0
12911,6LgaEHS0k29b7AGjgPwCRg,Round Up,10051 W Hillsborough Ave,Tampa,FL,33615,28.005005,-82.597306,3.0,17,0,"Nightlife, Bars, Country Dance Halls, Dance Clubs",0
70358,lhJIEFEQ0vjXCVIHCydwMQ,Jet Green Tree Services,NaN,Broomall,PA,19008,39.967933,-75.365635,4.5,5,1,"Tree Services, Home Services",0
62129,HTFSwE6jGnpbIrmLPH1mfA,The Artisan Exchange,"208 Carter Dr, Ste 13B",West Chester,PA,19382,39.948935,-75.587995,4.5,7,1,"Food, Bakeries, Farmers Market, Coffee & Tea",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,uPVKWy0VeMMHMJ0V87nMMw,Rolynos Original Pizza,3530 S Mann Rd,Indianapolis,IN,46221,39.775992,-86.249926,3.5,33,1,"Pizza, Restaurants",0
125992,m8Dz-UDgORUkIYwInLn0sQ,Sweet Jesus,5015 111 Street NW,Edmonton,AB,T6H 4M6,53.485639,-113.513688,3.0,15,0,"Food, Ice Cream & Frozen Yogurt",0
26263,tnJIyIXAyk__BkKKXW9XgQ,Sonoran Retractables,7045 N Camino Martin,Marana,AZ,85741,32.335229,-111.058103,2.5,5,1,"Home Services, Local Services, Awnings, Patio ...",0
104860,1hlmKs1lmW582mqjh3-MzQ,Hong Kong Sakura,1135 Hurffville Rd,Deptford,NJ,08097,39.835924,-75.087669,4.0,24,0,"Sushi Bars, Thai, Chinese, Restaurants, Japanese",0


# Vectorize categories

In [ ]:
# Get vector representations of genre
vec = CountVectorizer()
categories_vec = vec.fit_transform(business_df['categories'])

# Display resulting feature vectors
categories_vectorized = pd.DataFrame(categories_vec.todense(),columns=vec.get_feature_names_out(),index=business_df.business_id)
categories_vectorized.head()


,3d,abatement,acai,accessories,accountants,acne,active,activities,acupuncture,addiction,...,women,workers,wraps,yelp,yoga,yogurt,your,yourself,ziplining,zoos
business_id,,,,,,,,,,,,,,,,,,,,,
gczLFZVmpfsBB3uUHEbWiA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4n29_s1qDN5HQAYst74ClA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6LgaEHS0k29b7AGjgPwCRg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
lhJIEFEQ0vjXCVIHCydwMQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HTFSwE6jGnpbIrmLPH1mfA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Remove unnecessary categories (top 30 categories)

In [ ]:
csmatrix = cosine_similarity(categories_vec)
csmatrix = pd.DataFrame(csmatrix,columns=business_df.business_id,index=business_df.business_id)

In [ ]:
csmatrix.head(30)

business_id,gczLFZVmpfsBB3uUHEbWiA,4n29_s1qDN5HQAYst74ClA,6LgaEHS0k29b7AGjgPwCRg,lhJIEFEQ0vjXCVIHCydwMQ,HTFSwE6jGnpbIrmLPH1mfA,ryhPJ8yVzy7W_25lWOyPdg,5K6ljsbWjGzK7NcVMV41Nw,KpXHQMgHOZdS63WoYuV7ug,pyruq5Dnhn1Fteptkl-mUQ,HPOIKPq3nz28UcryuAVrWw,...,qtiPcQeweDuu6PxHH3GaCA,V5KiEoH4E86GA2S3RQkf1g,tr2_pKQRojSt_vEISfGApg,Dl9Ca8SED97_liUGAOuGdA,vRMnWct7c5hCvobYfIqrlw,uPVKWy0VeMMHMJ0V87nMMw,m8Dz-UDgORUkIYwInLn0sQ,tnJIyIXAyk__BkKKXW9XgQ,1hlmKs1lmW582mqjh3-MzQ,ZtLcDMt790gYR-ArV-u-zg
business_id,,,,,,,,,,,,,,,,,,,,,
gczLFZVmpfsBB3uUHEbWiA,1.000000,0.000000,0.000000,0.000000,0.154303,0.113961,0.000000,0.267261,0.000000,0.000000,...,0.000000,0.188982,0.241747,0.000000,0.000000,0.267261,0.169031,0.000000,0.154303,0.000000
4n29_s1qDN5HQAYst74ClA,0.000000,1.000000,0.000000,0.561951,0.000000,0.000000,0.000000,0.000000,0.095673,0.000000,...,0.076472,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.415029,0.000000,0.000000
6LgaEHS0k29b7AGjgPwCRg,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136083,0.000000
lhJIEFEQ0vjXCVIHCydwMQ,0.000000,0.561951,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.170251,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.615457,0.000000,0.000000
HTFSwE6jGnpbIrmLPH1mfA,0.154303,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.288675,0.000000,0.000000,...,0.000000,0.000000,0.261116,0.000000,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000
ryhPJ8yVzy7W_25lWOyPdg,0.113961,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.150756,0.000000,0.000000,0.000000,0.213201,0.000000,0.000000,0.123091,0.000000
5K6ljsbWjGzK7NcVMV41Nw,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.208514,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
KpXHQMgHOZdS63WoYuV7ug,0.267261,0.000000,0.000000,0.000000,0.288675,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.452267,0.000000,0.000000,0.000000,0.316228,0.000000,0.000000,0.000000
pyruq5Dnhn1Fteptkl-mUQ,0.000000,0.095673,0.000000,0.170251,0.000000,0.000000,0.208514,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125739,0.000000,0.000000


In [ ]:
nc_df = business_df[business_df["state"] == "FL"]
nc_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,open_past_10
12911,6LgaEHS0k29b7AGjgPwCRg,Round Up,10051 W Hillsborough Ave,Tampa,FL,33615,28.005005,-82.597306,3.0,17,0,"Nightlife, Bars, Country Dance Halls, Dance Clubs",0
116779,ryhPJ8yVzy7W_25lWOyPdg,Smokey Bones,2693 Gulf To Bay Blvd,Clearwater,FL,33759,27.960137,-82.726761,3.5,423,1,"American (New), American (Traditional), Burger...",0
41382,pyruq5Dnhn1Fteptkl-mUQ,Schooner Clearwater,"25 Causeway Blvd, Boatslip 20",Clearwater,FL,33767,27.976494,-82.827382,5.0,16,1,"Boating, Event Planning & Services, Hotels & T...",0
8724,HPOIKPq3nz28UcryuAVrWw,Insta Quick Oil Change,1724 S Missouri Ave,Clearwater,FL,33756,27.934130,-82.787910,4.5,10,1,"Oil Change Stations, Automotive",0
57650,F6xIsnKX5zotM4OsfYyGkQ,Pet Supermarket,5001 34th St S,St Petersburg,FL,33711,27.722033,-82.680697,3.0,10,1,"Pet Stores, Pets",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25511,okBgVNjxLFCQnVpR_PIIzw,Misred Outfitters,615 Central Ave,St. Petersburg,FL,33701,27.771385,-82.642086,4.5,14,1,"Accessories, Fashion, Jewelry, Women's Clothin...",0
40205,Ve97h3uXRrm3jAhJmIyTag,Boneyard,1823 E 7th Ave,Tampa,FL,33605,27.960082,-82.438332,3.5,72,1,"Bars, Chicken Wings, Restaurants, American (Tr...",0
102904,aZ2viI7tlQnoFGiuGL4M5A,Tower Radiology - Oldsmar,3870 Tampa Rd,Oldsmar,FL,34677,28.041186,-82.672004,3.5,8,1,"Doctors, Radiologists, Diagnostic Services, He...",0
33823,qtiPcQeweDuu6PxHH3GaCA,Goodwill Superstore,3365 Central Ave N,St. Petersburg,FL,33713,27.770997,-82.678673,2.5,18,0,"Used, Vintage & Consignment, Thrift Stores, Di...",0


In [ ]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,open_past_10
104262,gczLFZVmpfsBB3uUHEbWiA,HipCityVeg,121 S Broad St,Philadelphia,PA,19107,39.950265,-75.163685,4.5,157,1,"Vegan, Breakfast & Brunch, Vegetarian, Restaur...",0
65693,4n29_s1qDN5HQAYst74ClA,Always Open & Shut Garage Door Services,"4500 E Speedway Blvd, Ste 95",Tucson,AZ,85712,32.235482,-110.895939,4.0,28,1,"Contractors, Home Services, Shopping, Handyman...",0
12911,6LgaEHS0k29b7AGjgPwCRg,Round Up,10051 W Hillsborough Ave,Tampa,FL,33615,28.005005,-82.597306,3.0,17,0,"Nightlife, Bars, Country Dance Halls, Dance Clubs",0
70358,lhJIEFEQ0vjXCVIHCydwMQ,Jet Green Tree Services,NaN,Broomall,PA,19008,39.967933,-75.365635,4.5,5,1,"Tree Services, Home Services",0
62129,HTFSwE6jGnpbIrmLPH1mfA,The Artisan Exchange,"208 Carter Dr, Ste 13B",West Chester,PA,19382,39.948935,-75.587995,4.5,7,1,"Food, Bakeries, Farmers Market, Coffee & Tea",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,uPVKWy0VeMMHMJ0V87nMMw,Rolynos Original Pizza,3530 S Mann Rd,Indianapolis,IN,46221,39.775992,-86.249926,3.5,33,1,"Pizza, Restaurants",0
125992,m8Dz-UDgORUkIYwInLn0sQ,Sweet Jesus,5015 111 Street NW,Edmonton,AB,T6H 4M6,53.485639,-113.513688,3.0,15,0,"Food, Ice Cream & Frozen Yogurt",0
26263,tnJIyIXAyk__BkKKXW9XgQ,Sonoran Retractables,7045 N Camino Martin,Marana,AZ,85741,32.335229,-111.058103,2.5,5,1,"Home Services, Local Services, Awnings, Patio ...",0
104860,1hlmKs1lmW582mqjh3-MzQ,Hong Kong Sakura,1135 Hurffville Rd,Deptford,NJ,08097,39.835924,-75.087669,4.0,24,0,"Sushi Bars, Thai, Chinese, Restaurants, Japanese",0
